# Planning and Stock Report Generation

In [1]:
from utils import get_openai_api_key

import autogen

/home/muriloms/Documents/rep/ai-agent-projects/autogen/venv/lib/python3.11/site-packages/flaml/__init__.py:20: UserWarning: flaml.automl is not available. Please install flaml[automl] to enable AutoML functionalities.
  warnings.warn("flaml.automl is not available. Please install flaml[automl] to enable AutoML functionalities.")


In [2]:
OPEN_AI_KEY = get_openai_api_key()
llm_config={
    "model":"gpt-4o-mini"
}

### The task!

In [3]:
task = "Write a blogpost about the stock price performance of "\
"Nvidia in the past month. Today's date is 2024-12-23"

### Build a group chat
This group chat will include these agents:
1. **User_proxy** or **Admin**: to allow the user to comment on the report and ask the writer to refine it.
2. **Planner**: to determine relevant information needed to complete the task.
3. **Engineer**: to write code using the defined plan by the planner.
4. **Executor**: to execute the code written by the engineer.
5. **Writer**: to write the report.

In [4]:
user_proxy = autogen.ConversableAgent(
    name="Admin",
    system_message="Give the task, and send "
    "instrictions to writer to refine the blog post.",
    code_execution_config=False,
    llm_config=llm_config,
    human_input_mode="ALWAYS",
)

[autogen.oai.client: 01-16 14:21:07] {129} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


In [5]:
planner = autogen.ConversableAgent(
    name="Planner",
    system_message="Given a task, please determine "
    "what information is needed to complete the task. "
    "Please note that the information will all be retrieved using"
    " Python code. Please only suggest information that can be "
    "retrieved using Python code. "
    "After each step is done by others, check the progress and "
    "instruct the remaining steps. If a step fails, try to "
    "workaround",
    description="Planner. Given a task, determine what "
    "information is needed to complete the task. "
    "After each step is done by others, check the progress and "
    "instruct the remaining steps",
    llm_config=llm_config,
)

[autogen.oai.client: 01-16 14:23:22] {129} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


In [6]:
engineer = autogen.AssistantAgent(
    name="Engineer",
    llm_config=llm_config,
    description="An engineer that writes code based on the plan "
    "provided by the planner.",
)

[autogen.oai.client: 01-16 14:23:43] {129} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


In [7]:
executor = autogen.ConversableAgent(
    name="Executor",
    system_message="Execute the code written by the "
    "engineer and report the result.",
    human_input_mode="NEVER",
    code_execution_config={
        "last_n_messages": 3,
        "work_dir": "coding",
        "use_docker": False,
    },
)

In [8]:
writer = autogen.ConversableAgent(
    name="Writer",
    llm_config=llm_config,
    system_message="Writer."
    "Please write blogs in markdown format (with relevant titles)"
    " and put the content in pseudo ```md``` code block. "
    "You take feedback from the admin and refine your blog.",
    description="Writer."
    "Write blogs based on the code execution results and take "
    "feedback from the admin to refine the blog."
)

[autogen.oai.client: 01-16 14:24:14] {129} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


### Define the group chat

In [9]:
groupchat = autogen.GroupChat(
    agents=[user_proxy, engineer, writer, executor, planner],
    messages=[],
    max_round=10,
)

In [10]:
manager = autogen.GroupChatManager(
    groupchat=groupchat, llm_config=llm_config
)

[autogen.oai.client: 01-16 14:26:29] {129} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


### Start the group chat!

In [11]:
groupchat_result = user_proxy.initiate_chat(
    manager,
    message=task,
)

Admin (to chat_manager):

Write a blogpost about the stock price performance of Nvidia in the past month. Today's date is 2024-12-23

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
Planner (to chat_manager):

To write a blog post about the stock price performance of Nvidia over the past month, the following information is needed and can be retrieved using Python code:

1. **Current Stock Price**: The latest stock price of Nvidia.
2. **Historical Stock Prices**: The stock prices of Nvidia for the past month. This includes the closing price from 2024-11-22 to 2024-12-22.
3. **Price Change**: Calculate the change in stock price over the past month (including percentage change).
4. **Trading Volume**: The trading volume of Nvidia stocks over the past month to understand market activity.
5. **Key News/Events**: Any significant news or developments related to Nvidia that may have affected the stock price during the past month.



### Add a speaker selection policy

In [12]:
user_proxy = autogen.ConversableAgent(
    name="Admin",
    system_message="Give the task, and send "
    "instructions to writer to refine the blog post.",
    code_execution_config=False,
    llm_config=llm_config,
    human_input_mode="ALWAYS",
)

planner = autogen.ConversableAgent(
    name="Planner",
    system_message="Given a task, please determine "
    "what information is needed to complete the task. "
    "Please note that the information will all be retrieved using"
    " Python code. Please only suggest information that can be "
    "retrieved using Python code. "
    "After each step is done by others, check the progress and "
    "instruct the remaining steps. If a step fails, try to "
    "workaround",
    description="Given a task, determine what "
    "information is needed to complete the task. "
    "After each step is done by others, check the progress and "
    "instruct the remaining steps",
    llm_config=llm_config,
)

engineer = autogen.AssistantAgent(
    name="Engineer",
    llm_config=llm_config,
    description="Write code based on the plan "
    "provided by the planner.",
)

writer = autogen.ConversableAgent(
    name="Writer",
    llm_config=llm_config,
    system_message="Writer. "
    "Please write blogs in markdown format (with relevant titles)"
    " and put the content in pseudo ```md``` code block. "
    "You take feedback from the admin and refine your blog.",
    description="After all the info is available, "
    "write blogs based on the code execution results and take "
    "feedback from the admin to refine the blog. ",
)

executor = autogen.ConversableAgent(
    name="Executor",
    description="Execute the code written by the "
    "engineer and report the result.",
    human_input_mode="NEVER",
    code_execution_config={
        "last_n_messages": 3,
        "work_dir": "coding",
        "use_docker": False,
    },
)

[autogen.oai.client: 01-16 14:30:04] {129} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.
[autogen.oai.client: 01-16 14:30:05] {129} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.
[autogen.oai.client: 01-16 14:30:05] {129} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.
[autogen.oai.client: 01-16 14:30:05] {129} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


In [13]:
groupchat = autogen.GroupChat(
    agents=[user_proxy, engineer, writer, executor, planner],
    messages=[],
    max_round=10,
    allowed_or_disallowed_speaker_transitions={
        user_proxy: [engineer, writer, executor, planner],
        engineer: [user_proxy, executor],
        writer: [user_proxy, planner],
        executor: [user_proxy, engineer, planner],
        planner: [user_proxy, engineer, writer],
    },
    speaker_transitions_type="allowed",
)

In [14]:
manager = autogen.GroupChatManager(
    groupchat=groupchat, llm_config=llm_config
)

groupchat_result = user_proxy.initiate_chat(
    manager,
    message=task,
)

[autogen.oai.client: 01-16 14:30:32] {129} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.
Admin (to chat_manager):

Write a blogpost about the stock price performance of Nvidia in the past month. Today's date is 2024-12-23

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
Writer (to chat_manager):

```md
# Nvidia's Stock Price Performance: A Review of the Past Month

As we close out 2023, Nvidia Corporation (NASDAQ: NVDA) has once again captured the attention of investors and analysts alike. Over the past month, Nvidia's stock price has exhibited remarkable fluctuations, reflecting broader trends in the technology and semiconductor sectors as well as the company's ongoing developments in AI and computing technologies.

## Key Performance Metrics

At the beginning of December, Nvidia's stock was trading around **$420 per share**. Fast forward to late December, and the